# Making a script

One of the things that dissapoints me most about OpenAI's Whisper is that the resulting text is just one long wall of text.  Fortunately, we are using a discord bot called Craig to record our DnD sessions.  With Craig, each person's voice is recorded to their own audio file, which means that the audio is already diarized.

Therefore it stands to reason that in order to generate a text file in a script format, all I would need to do is process each file individually then order the segments in chronological order.  Each time the speaker changes simply insert a line with that person's name.

In [ ]:
import os, re
import whisper
import torch
import torchaudio as ta
import librosa
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
from IPython.display import Audio, display

print(whisper.__version__)
print(ta.__version__)
print(torch.__version__)
%matplotlib inline

## Parsing Audacity projects
The method we've been using with Craig is to download our recordings as an audacity project.  Audacity projects have a `project.aup` file, which is just a text file that tells the program how to arrange the audio files used in the project.  Let's take a look.

In [ ]:
data_dir = '../data/db_02-03-2023/'
aup_file = 'V8DcTChKFF.aup'

with open(os.path.join(data_dir, aup_file)) as f:
    aup = BeautifulSoup(f, features="lxml-xml")

# Project metadata
project = aup.find('project')
pprint(project.attrs)


`projname` tells us the directory in which the audio files are stored.

In [ ]:
proj_data_dir = project.get('projname')

proj_data_dir

In [ ]:
p = os.path.join(data_dir, proj_data_dir)
os.listdir(p)

4 audio files and two data files

In [ ]:
proj_imports = project.find_all('import')
proj_imports

Here there's two attributes that seem immediately useful.  `filename` tells us where the audio is.  `offset` tells us how each file aligns with the others in the project.

In [ ]:
discord_name_pattern = r'.+-(.*)\..+'
proj_files = []

for item in proj_imports:
    filename, offset= item.get('filename'), float(item.get('offset'))
    username = re.search(discord_name_pattern, filename).group(1)
    out = {
        'filename': filename,
        'username': username,
        'offset': offset
    }
    proj_files.append(out)

pprint(proj_files)

In [ ]:
whisper_dir = '../models/whisper'
transcriber =   whisper.load_model('small.en', download_root= whisper_dir)

In [ ]:
for part in proj_files[:1]:
    filename = part.get('filename')
    username = part.get('username')
    offset = part.get('offset')

    path = os.path.join(data_dir, proj_data_dir, filename)
    wav = whisper.load_audio(path)

wav

In [ ]:
# condition_on_previous_text=False made this run in 12 minutes.  Without the argument it took 21 minutes.
results = transcriber.transcribe(wav, condition_on_previous_text=False)

In [ ]:
for thing in results['segments']:
    print(f"[{thing['start']} - {thing['end']}] {thing['text']}")

Hmm.. so Whisper is generating a lot of repetitive text.  This is likely due to the fact that the audio is very long with long sections of silence.  However, I'm noticing that the timestamps presented are much more accurate than normal.  Likely another benifit of setting `condition_on_previous_text=False`

I'm wondering if using Siler VAD to strip out the non-speech will improve transcription accuracy.

In [ ]:
silero_dir = '../models/silero-vad'
vad, utils =    torch.hub.load(repo_or_dir=silero_dir,
                               source='local',
                               model='silero_vad',
                               force_reload=True,
                               onnx=False)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

In [ ]:
speech_timestamps = get_speech_timestamps(wav, vad)

In [ ]:
len(speech_timestamps)

In [ ]:
SAMPLING_RATE = 16000
speech_only = collect_chunks(speech_timestamps, torch.tensor(wav))

In [ ]:
Audio(data=speech_only, rate=SAMPLING_RATE)

In [ ]:
# 6m 46s!
results = transcriber.transcribe(speech_only, condition_on_previous_text=False)


In [ ]:
print(results['text'])

In [ ]:
for thing in results['segments']:
    print(f"[{thing['start']} - {thing['end']}] {thing['text']}")

In [ ]:
len(results['segments'])

In [ ]:
for segment in results['segments']:
    segment['start_frame'] = segment['start'] * SAMPLING_RATE
    segment['end_frame'] = segment['end'] * SAMPLING_RATE
    segment['length'] = segment['end_frame'] - segment['start_frame']

In [ ]:
results['segments'][0:2]

In [ ]:
speech_ts_df = pd.DataFrame(speech_timestamps)
speech_ts_df['length'] = speech_ts_df['end'] - speech_ts_df['start']
speech_ts_df.sort_values(by='start')

In [ ]:
speech_timestamps[-1]

In [ ]:
len(speech_timestamps)

In [ ]:
chunk_timestamps = []
current_frame = 0
for i, entry in enumerate(speech_timestamps):
    speech_length = entry['end'] - entry['start']
    end_frame = current_frame + speech_length
    chunk_timestamps.append(
        {'start': current_frame,
         'end': end_frame
        }
        )
    current_frame = end_frame+1

len(chunk_timestamps)

In [ ]:
a = 1
b = speech_timestamps[a]
c = chunk_timestamps[a]
d,e = b['start'], b['end']
f,g = c['start'], c['end']

display(Audio(wav[d:e], rate=SAMPLING_RATE))
display(Audio(speech_only[f:g], rate=SAMPLING_RATE))

In [ ]:
a=450
i = results['segments'][a]
start = round(i['start'] * 16000)
end = round(i['end'] * 16000)
print(start, i['text'], end)
display(Audio(speech_only[start:end], rate=SAMPLING_RATE))

In [ ]:
len(results['segments'])

## Time to use pandas

The timestamps provided by Whisper become slightly out of sync with the timestamps provided by Silero.  This means that I can't match Whisper segments directly with Silero segments.  
However, the drift doesn't seem too bad and is likely caused by the way that Whisper is converting the frame timestamps to seconds, so it should be sufficient to just find the closest ones.

By putting both the long waveform timestamps and the truncated chunk timestamps into the same dataframe I can easily match the two together with a `segment_id` identifying them as the same segment of audio.  Additionally, having the timestamps sorted will make it easier to match Silero segments with Whisper segments.

In [ ]:
vad_df = pd.DataFrame(speech_timestamps)
vad_df['length'] = vad_df['end'] - vad_df['start']
vad_df.head()

In [ ]:
tdf = pd.DataFrame(chunk_timestamps)
tdf.columns = ['chunk_start','chunk_end']
tdf['chunk_length'] = tdf['chunk_end'] - tdf['chunk_start']
tdf.head()

In [ ]:
vad_df = pd.concat([vad_df, tdf], axis=1)

In [97]:
vad_df['id'] = vad_df.index
vad_df.head()

,start,end,length,chunk_start,chunk_end,chunk_length,id
0,2080,17376,15296,0,15296,15296,0
1,363552,382432,18880,15297,34177,18880,1
2,384544,416224,31680,34178,65858,31680,2
3,425504,444896,19392,65859,85251,19392,3
4,538144,566752,28608,85252,113860,28608,4


In [85]:
w_df = pd.DataFrame(results['segments'])
w_df.head()

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob,start_frame,end_frame,length
0,0,0,0.00,1.00,Now recording.,"[2735, 8296, 13]",0.0,-0.446743,1.57931,0.595192,0.0,16000.0,16000.0
1,1,0,1.00,4.72,"You know Josh, he could have just right click...","[921, 760, 8518, 11, 339, 714, 423, 655, 826, ...",0.0,-0.446743,1.57931,0.595192,16000.0,75520.0,59520.0
2,2,0,4.72,5.72,too.,"[1165, 13]",0.0,-0.446743,1.57931,0.595192,75520.0,91520.0,16000.0
3,3,0,5.72,7.22,"Anyway, welcome.","[21836, 11, 7062, 13]",0.0,-0.446743,1.57931,0.595192,91520.0,115520.0,24000.0
4,4,0,7.22,8.72,How would he be able to tell me?,"[1374, 561, 339, 307, 1498, 284, 1560, 502, 30]",0.0,-0.446743,1.57931,0.595192,115520.0,139520.0,24000.0


In [105]:
display(w_df.loc[2:2])

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob,start_frame,end_frame,length
2,2,0,4.72,5.72,too.,"[1165, 13]",0.0,-0.446743,1.57931,0.595192,75520.0,91520.0,16000.0


In [110]:
w_df.loc[2].start

4.72

In [127]:
a = 0

In [137]:
a -= 1
display(w_df.loc[a:a][['start_frame','end_frame','length','text','id']])
w_start = w_df.loc[a]['start_frame']
w_end = w_df.loc[a]['end_frame']
start_segment = vad_df.iloc[(vad_df['chunk_start']-w_start).abs().argsort()][:1]
end_segment = vad_df.iloc[(vad_df['chunk_end']-w_end).abs().argsort()][:1]
tdf = pd.concat([start_segment, end_segment])
display(tdf)

display(Audio(speech_only[tdf.iloc[0].chunk_start: tdf.iloc[1].chunk_end], rate=SAMPLING_RATE))

,start_frame,end_frame,length,text,id
6,174720.0,190720.0,16000.0,"Oh, below.",6


,start,end,length,chunk_start,chunk_end,chunk_length,id
7,1220640,1233888,13248,174151,187399,13248,7
7,1220640,1233888,13248,174151,187399,13248,7
